# 🕮 9: 魔法方法、特性和迭代器


🖊本章将讨论几个重要的魔法方法以及两个相关主题：特性和迭代器。

* 9.1 [如果你使用的不是 Python 3 ](#9.1-如果你使用的不是Python3)
* 9.2 [构造函数 ](#9.2-构造函数)
* 9.3 [元素访问](#9.3-元素访问)
* 9.4 [其他魔法方法]()
* 9.5 [特性](#9.5-特性)
* 9.6 [迭代器](#9.6-迭代器)
* 9.7 [生成器](#9.6-生成器)
* 9.8 [八皇后问题 ](#9.8-八皇后问题 )

## 9.1 如果你使用的不是Python3

Python 2中新式类要在模块开头包含赋值语句`__metaclass__ = type`

In [0]:
#__metaclass__ = type
class NewStyle(object):     
  more_code_here 
class OldStyle:    
  more_code_here 
  
#书中并没有在所有示例中都显式地设置元类或继承object

## 9.2 构造函数
 
 命名为`__init__` 在对象创建后自动调用。
 
 用得最多的**魔法方法**...

In [1]:
class FooBar:     
  def __init__(self, value=42):  
    self.somevar = value
f = FooBar()
print(f.somevar)
g = FooBar('This is a constructor argument') 
g.somevar

42


'This is a constructor argument'



---


**注意 ** *Python提供了魔法方法`__del__`，也称作析构函数（destructor）。这个方法在对象被销毁 （作为垃圾被收集）前被调用，但鉴于无法知道准确的调用时间，建议尽可能不要使 用`__del__`。 *

---

### 9.2.1 重写普通方法和特殊的构造函数 

In [2]:
class A:     
  def hello(self):  
    print("Hello, I'm A.") 
# class B(A):    
#   pass 
class B(A):    
  def hello(self):  
    print("Hello, I'm B.") 

a = A()
b = B()
a.hello()
b.hello()

Hello, I'm A.
Hello, I'm B.


* 重写**构造函数**必须调用**超类**的构造函数

In [3]:
class Bird:     
  def __init__(self):     
    self.hungry = True    
  def eat(self):
    if self.hungry:   
      print('Aaaah ...')   
      self.hungry = False  
    else:   
      print('No, thanks!')

b = Bird() 
b.eat()
b.eat()

Aaaah ...
No, thanks!


In [4]:
class SongBird(Bird):     
  def __init__(self):     
    self.sound = 'Squawk!'    
  def sing(self): 
    print(self.sound)
    
sb = SongBird() 
sb.sing()
sb.eat()#重新了构造函数，不包含属性hungry的代码

Squawk!


AttributeError: 'SongBird' object has no attribute 'hungry'

* 调用未关联的超类构造函数 

In [5]:
class SongBird(Bird):     
  def __init__(self):     
    Bird.__init__(self) #使用超类的构造函数初始化SongBird对象
    self.sound = 'Squawk!'    
  def sing(self): 
    print(self.sound)
    
sb = SongBird() 
sb.sing()
sb.eat()#重新了构造函数，不包含属性hungry的代码

Squawk!
Aaaah ...


* 使用函数super **常用**

In [6]:
class SongBird(Bird):     
  def __init__(self):     
    super().__init__()  #使用super函数初始化SongBird对象
    self.sound = 'Squawk!'    
  def sing(self): 
    print(self.sound)
    
sb = SongBird() 
sb.sing()
sb.eat()#重新了构造函数，不包含属性hungry的代码

Squawk!
Aaaah ...


## 9.3 元素访问

创建行为类似于序列或映射的对象

* ` __len__(self)`: 返回集合中的项数
* `__getitem__(self, key)`：指定键关联的值
* `__setitem__(self, key, value)`:设定指定键值
* `__delitem__(self, key)`：删除指定项

* 从 `list`、`dict`、`str`派生


In [11]:
class CounterList(list):     
  def __init__(self, *args):  
    super().__init__(*args)     
    self.counter = 0    
  def __getitem__(self, index):  
    self.counter += 1  
    return super().__getitem__(index) #super(CounterList, self).__getitem__(index)  

In [12]:
cl = CounterList(range(10)) 
cl
cl.reverse()
print(cl)
cl.counter
cl[4] + cl[2]
cl.counter

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


2

## 9.5 特性

Python中隐藏的存取方法即为特性（property）。 


In [13]:
class Rectangle:     
  def __init__(self):  
    self.width = 0     
    self.height = 0    
  def set_size(self, size):  
    self.width, self.height = size    
  def get_size(self):  
    return self.width, self.height 

In [14]:
r = Rectangle()
r.width = 10
r.heigth = 5
r.get_size()
r.set_size((150, 100))
r.get_size()

(150, 100)

* 函数`property`

 `property(get_size, set_size)`  #获取方法在前，设定方法在后
 
 `property(fget, fset,  fdel, doc）` 可不指定参数，或用关键字参数指定特定参数
 

In [16]:
class Rectangle:     
  def __init__(self):  
    self.width = 0     
    self.height = 0    
  def set_size(self, size):  
    self.width, self.height = size    
  def get_size(self):  
    return self.width, self.height 
  size = property(get_size, set_size) #

In [19]:
r = Rectangle()
r.width = 10
r.heigth = 5
#r.size
r.size = 150, 100 # 看起来像普通属性一样
r.width

150

***对于新式类应使用特性而不是存取方法***

* 静态方法和类方法 

静态方法没有参数self、类方法包含类似self的参数，通常被命名为cls


In [20]:
class MyClass: 
  @staticmethod     #装饰器
  def smeth():  
    print('This is a static method') 
  @classmethod    
  def cmeth(cls):  
    print('This is a class method of', cls)

In [21]:
MyClass.smeth() 
MyClass.cmeth() 

This is a static method
This is a class method of <class '__main__.MyClass'>


###  **`__getattr__`、`__setattr__`**等方法 

*  `__getattribute__(self, name)``：在属性被访问时自动调用（只适用于新式类）。

* `__getattr__(self, name)`：在属性被访问而对象没有这样的属性时自动调用。 

*  `__setattr__(self, name, value)`：试图给属性赋值时自动调用。 

*  `__delattr__(self, name)`：试图删除属性时自动调用。


In [24]:
class Rectangle:     
  def __init__ (self):  
    self.width = 0     
    self.height = 0    
  def __setattr__(self, name, value):  
    if name == 'size':   
      self.width, self.height = value  
    else:   
      self. __dict__[name] = value    
  def __getattr__(self, name):  
    if name == 'size':   
        return self.width, self.height  
    else:   
      raise AttributeError()

* 即便涉及的属性不是size，也将调用方法`__setattr__`
* 仅当没有找到指定的属性时，才会调用方法`__getattr__`



---



## 9.6 迭代器

魔法方法`__iter__` 是迭代器的基础


In [25]:
#使用迭代器而不是使用列表
class Fibs:     
  def __init__(self):  
    self.a = 0     
    self.b = 1    
  def __next__(self):  
    self.a, self.b = self.b, self.a + self.b  
    return self.a    
  def __iter__(self):     
    return self

In [37]:
fibs = Fibs()

for f in fibs:
  print(f)
  if f > 1000:
    print(f)
    break

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
1597


In [28]:
it = iter([1, 2, 3])
next(it)
#next(it)

1

## 9.7 生成器

`yield`返回一个**生成器**
  
  可以使代码很优雅但完全可以不使用。

In [44]:
nested = [[1, 2], [3, 4], [5], [2] ] 
#展开嵌套列表
def flatten(nested):  
  for sublist in nested:  
    for element in sublist:
      yield element #包含yield语句的函数都被称为生成器
      #print (element)
for num in flatten(nested):
  print(num)

1
2
3
4
5
2


TypeError: 'int' object is not iterable

生成器不使用return返回一个值，而是可以生成多个值，每次一个。每次使用yield生成一个值后，函数都将冻结，即在此停止执行，等待被重新唤醒。被重新唤醒后，函数将从停止的地方开始继续执行

In [45]:
def flatten(nested):    
  try:  
    for sublist in nested:   
      for element in flatten(sublist):    
        yield element    
  except TypeError:  
    yield nested
list(flatten([[[1], 2], 3, 4, [5, [6, 7]], 8])) 

[1, 2, 3, 4, 5, 6, 7, 8]

In [46]:
def flatten(nested):    
  try: 
     # 不迭代类似于字符串的对象：  
    try: nested + ''  
    except TypeError: pass  
    else: raise TypeError 
    for sublist in nested:   
      for element in flatten(sublist):    
        yield element    
  except TypeError:  
    yield nested
    
list(flatten(['foo', ['bar', ['baz']]])) 

['foo', 'bar', 'baz']

`yield`意味着应生成一个值，而`return`意味着生成器应停止执行

生成器由两个单独的部分组成：生成器的函数和生成器的迭代器。生成器的函数是由def语句定义的，其中包含yield。生成器的迭代器是这个函数返回的结果。用不太准确的话说，这两个实体通常被视为一个，通称为生成器。

In [60]:
def simple_generator():   
  yield 1 
    
simple_generator
simple_generator()

<generator object simple_generator at 0x7f20043d4ca8>

In [57]:
def repeater(value):  
  while True:  
    new = (yield value)  
    if new is not None: value = new 
r = repeater(42)
next(r)
r.send("Hello, world!") 
r.send(1)
print(next(r))
print(next(r))

1
1


**外部世界**：外部世界可访问生成器的方法`send`，这个方法类似于`next`，但接受一个参数（要发送的“消息”，可以是任何对象）。 

**生成器**：在挂起的生成器内部，yield可能用作表达式而不是语句。换而言之，当生成器重新运行时，yield返回一个值——通过`send`从外部世界发送的值。如果使用的是`next`，` yield`将返回None。 
 
**方法throw**：用于在生成器中（yield表达式处）引发异常，调用时可提供一个异常类型、一 个可选值和一个traceback对象。 

**方法close**：用于停止生成器，调用时无需提供任何参数。 

## 9.8 八皇后问题

In [63]:
import random
def conflict(state, nextX):   
  nextY = len(state)  
  for i in range(nextY):  
    if abs(state[i] - nextX) in (0, nextY - i):   
      return True    
    return False

def prettyprint(solution):    
  def line(pos, length=len(solution)):     
    return '. ' * (pos) + '👑 ' + '. ' * (length-pos-1)    
  for pos in solution:  
    print(line(pos))
  
  
def queens(num=8, state=()):     
  for pos in range(num):  
    if not conflict(state, pos):  
      if len(state) == num-1:     
        yield (pos,)  
      else:  
        for result in queens(num, state + (pos,)):   
          yield (pos,) + result 
# for solution in queens(4): 
#   print(solution)

prettyprint(random.choice(list(queens(8)))) 

. . . . . . . 👑 
. . . 👑 . . . . 
👑 . . . . . . . 
👑 . . . . . . . 
. . . . . . 👑 . 
. . . 👑 . . . . 
👑 . . . . . . . 
. . . . . 👑 . . 
